<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/tweet_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
!pip install xlrd
import xlrd
import re
import pickle
import csv

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [2]:
# Uncomment if you're running it for the first time
!pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/tathagata.raha/anaconda/envs/p3/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
# Uncomment if you're running it for the first time
!pip install tweet-preprocessor
import preprocessor as tweet_proc

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [4]:
# Uncomment if you're running it for the first time
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


#### *Raw Datasets are hosted [here](https://drive.google.com/drive/folders/1TuHRQQ41lK9oXJhlhsRiMjMnczuKn_kF?usp=sharing).*

In [5]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [6]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [7]:
is_hindi = 1
# change to test while running on the test set
datatype = "test"
# location of the tsv file
file_name = "../data/"+datatype+".tsv"
dest = '../temp/preprocessed/'

In [8]:
# Initializing Lists
datapoints_count = 0
see_index = True

tweets = []
raw_tweet_texts = []
tokenized_tweets = []
hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []
clean = []
task_1_labels = []
task_2_labels = []
tweet_ids = []
hasoc_ID = []

In [9]:
def strip_list(listie):
  stripped = []
  for item in listie:
    stripped.append(item.strip())
  return stripped

def hindi_clean(line, parse_obj):
  # beta
  tokens = line.replace(":", " : ").replace(",", " , ").replace(";", " ; ").split(" ")
  valid_stri = ""

  for raw_token in tokens:
    token = raw_token.strip()
    if token in strip_list(make_list(parse_obj.hashtags)):
      continue
    if token in strip_list(make_list(parse_obj.smileys)):
      continue
    if token in strip_list(make_list(parse_obj.emojis)):
      continue
    if token in strip_list(make_list(parse_obj.urls)):
      continue
    if token in strip_list(make_list(parse_obj.mentions)):
      continue
    if token in strip_list(make_list(parse_obj.numbers)):
      continue
    if token in strip_list(make_list(parse_obj.reserved)):
      continue
    valid_stri = valid_stri + " " + token
  return valid_stri.strip()

In [10]:
if datatype == 'train' or datatype=='valid':
#     workbook = xlrd.open_workbook(file_name)
#     sheet = workbook.sheet_by_index(0)

#     for row in range(sheet.nrows):
#         line = sheet.row_values(row)

    file = open(file_name, 'r')
    count = 0
    file_reader = csv.reader(file, delimiter = "\t", quoting=csv.QUOTE_NONE)
    for line in file_reader:
#         if see_index == True:
#             see_index = False
#             continue

        datapoints_count += 1
        tweet_ids.append(line[0])
        task_1_labels.append(line[2])
        tweets.append(line[1].replace("\n", " "))
        temp = line[1].replace("\n", " ")
        parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
        tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
        tag = strip_list(make_list(parse_obj.hashtags))
        hashtags.append(tag)
        smiley = strip_list(make_list(parse_obj.smileys))
        smileys.append(smiley)
        emoji = strip_list(make_list(parse_obj.emojis))
        emojis.append(emoji)
        url = strip_list(make_list(parse_obj.urls))
        urls.append(url)
        mention = strip_list(make_list(parse_obj.mentions))
        mentions.append(mention)
        numbers.append(strip_list(make_list(parse_obj.numbers)))
        reserveds.append(strip_list(make_list(parse_obj.reserved)))
        for i in url + mention + smiley + emoji + tag:
#             print(i, tem)
            temp = temp.replace(i, " ")
        clean.append(temp)
        if is_hindi == 0:
          raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
        else:
          raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

    print("Number of Datapoints: " + str(datapoints_count))

In [11]:
datapoints_count

0

In [12]:
if datatype == 'test':
    file = open(file_name, 'r')
    file_reader = csv.reader(file, delimiter = "\t", quoting=csv.QUOTE_NONE)
    for line in file_reader:
        if see_index == True:
            see_index = False
            continue
#         print(line)
        datapoints_count += 1
        tweet_ids.append(line[0])
#         task_1_labels.append(line[2])
#         task_2_labels.append(line[3])
#         hasoc_ID.append(line[4])
        tweets.append(line[1].replace("\n", " "))

        temp = line[1].replace("\n", " ")
        parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
        tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
        tag = strip_list(make_list(parse_obj.hashtags))
        hashtags.append(tag)
        smiley = strip_list(make_list(parse_obj.smileys))
        smileys.append(smiley)
        emoji = strip_list(make_list(parse_obj.emojis))
        emojis.append(emoji)
        url = strip_list(make_list(parse_obj.urls))
        urls.append(url)
        mention = strip_list(make_list(parse_obj.mentions))
        mentions.append(mention)
        numbers.append(strip_list(make_list(parse_obj.numbers)))
        reserveds.append(strip_list(make_list(parse_obj.reserved)))
        for i in url + mention + smiley + emoji + tag:
#             print(i, tem)
            temp = temp.replace(i, " ")
        clean.append(temp)
        if is_hindi == 0:
          raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
        else:
          raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

    print("Number of Datapoints: " + str(datapoints_count))

Number of Datapoints: 1653


In [13]:
# Viewing Created Dataset

print("Tweets:")
print(tweets[0: 5])

print("Raw Texts:")
print(raw_tweet_texts[0: 5])

print("Hashtags:")
print(hashtags[0: 5])

print("Smileys:")
print(smileys[0: 5])

print("Emojis:")
print(emojis[0: 5])

print("Urls:")
print(urls[0: 5])

print("Mentions:")
print(mentions[0: 5])

print("Numbers:")
print(numbers[0: 5])

print("Reserved Words:")
print(reserveds[0: 5])
print("Cleaned")
print(clean[0:5])
print("Task Labels:")
print(task_1_labels[0: 5])
# print(task_2_labels[0: 5])

Tweets:
['कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नहीं मिलता है 20 करोड को रोजगार दे दिया है वह भी मात्र 6 साँल में चार साल अभी बाकी है और हर साल दो करोड़ रोजगार देने का ही वादा था 10 साल में देना था 20 करोड को लोगो को रोजगार जो मात्र 6 साल में लक्ष्य को प्राप्त करने वाली पहली सरकार है', 'पटना: BMP कैंप में पुरुष और महिला कांस्टेबल ने गोली मारकर की खुदकुशी, जांच में जुटी पुलिस  @kumarprakash4u की रिपोर्ट  https://t.co/Dq05hREifM', 'कोई भी कांग्रेसी, ऊंची छत पर, रेलवे लाइन पर, ऊंची बिल्डिंग पर, एकांत जगह पर, कुए के पास दिखाई दे, तुरंत पुलिस को सूचित करें,🙏 😂👍 विलुप्त होती हुई प्रजातियो को बचाना हमारा फर्ज है।', 'अंडरवर्ल्ड डॉन छोटा राजन के भाई को बीजेपी द्वारा टिकट मिला है।  ', 'RT @_Pb_swain_: इन पंचर छापों को कोन समझाए कि उनके रोजगार में कमी का कारण मोदी नहीं  👇 ट्यूब लैस टायर है.😂😂😂😂']
Raw Texts:
['कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नहीं मिलता है करोड को रोजगार दे दिया है वह भी मात्र साँल में चार साल अभी बाकी है और हर साल दो करोड़ रोजगार देने का ही वादा था साल में देना थ

In [14]:
# Generating Emoji Texts
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text.replace("_", " "))
  emoji_texts.append(texts)

print("Emoji Descriptions:")
print(emoji_texts[0: 5])

Emoji Descriptions:
[[], [], ['folded hands', 'face with tears of joy', 'thumbs up'], [], ['backhand index pointing down', 'face with tears of joy', 'face with tears of joy', 'face with tears of joy', 'face with tears of joy']]


In [15]:
# Segmenting Hashtags
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

print("Segmented Hashtags:")
print(segmented_hashtags[0: 5])

Segmented Hashtags:
[[], [], [], [], []]


In [16]:
name = dest + datatype+'.pickle'
dickie = {}
dickie['tweet_id'] = tweet_ids
if datatype != 'test':
    dickie['task_1'] = task_1_labels
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags
dickie['clean'] = clean
with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [17]:
with open(name, 'rb') as f:
  try_dict = pickle.load(f)

sizes = []
for key in try_dict.keys():
  sizes.append(len(try_dict[key]))

# Verifying if all sizes are equal
print(sizes)

[1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653, 1653]


In [18]:
# ^_^ Thank You